# A04_LLaMA-Factory-Cmd-Check_model_size

## 初始環境設定
- 重新reload kernel, 並切換到 kernel Image_S-work-LlamaFactory_c00cjz00_cuda11.8_pytorch_2.1.2-cuda11.8-cudnn8-devel

In [1]:
# 初始環境設定
import os
from pathlib import Path
HOME = str(Path.home())
Add_Binarry_Path=HOME+'/.local/bin'
os.environ['PATH']=os.environ['PATH']+':'+Add_Binarry_Path

In [2]:
# GPU確認
!nvidia-smi
import torch
torch.cuda.is_available()

Tue Mar 12 07:56:32 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:1B:00.0 Off |                    0 |
| N/A   57C    P0   216W / 300W |  14914MiB / 32768MiB |     94%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  Off  | 00000000:3D:00.0 Off |                    0 |
| N/A   

True

In [3]:
# 套件安裝 (此映像檔已經安裝以下套件, 若使用其他映像檔, 請移除 ## 安裝以下套件)
##!pip install llmtuner==0.5.3 deepspeed==0.13.1  bitsandbytes==0.42.0 -q 
##!pip install deepspeed==0.13.1 -q
##!pip install bitsandbytes==0.42.0 -q

## Deepspeed Stage2, 7B

In [4]:
from transformers import AutoModel
from deepspeed.runtime.zero.stage_1_and_2 import estimate_zero2_model_states_mem_needs_all_live
model = AutoModel.from_pretrained("/work/u00cjz00/slurm_jobs/github/models/Llama-2-7b-chat-hf")
estimate_zero2_model_states_mem_needs_all_live(model, num_gpus_per_node=1, num_nodes=1)
estimate_zero2_model_states_mem_needs_all_live(model, num_gpus_per_node=2, num_nodes=1)
estimate_zero2_model_states_mem_needs_all_live(model, num_gpus_per_node=4, num_nodes=1)
estimate_zero2_model_states_mem_needs_all_live(model, num_gpus_per_node=8, num_nodes=1)

[2024-03-12 07:56:40,172] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Estimated memory needed for params, optim states and gradients for a:
HW: Setup with 1 node, 1 GPU per node.
SW: Model with 6607M total params.
  per CPU  |  per GPU |   Options
  147.69GB |  12.31GB | offload_optimizer=cpu 
   36.92GB | 123.07GB | offload_optimizer=none
Estimated memory needed for params, optim states and gradients for a:
HW: Setup with 1 node, 2 GPUs per node.
SW: Model with 6607M total params.
  per CPU  |  per GPU |   Options
  147.69GB |  12.31GB | offload_optimizer=cpu 
   73.84GB |  73.84GB | offload_optimizer=none
Estimated memory needed for params, optim states and gradients for a:
HW: Setup with 1 node, 4 GPUs per node.
SW: Model with 6607M total params.
  per CPU  |  per GPU |   Options
  147.69GB |  12.31GB | offload_optimizer=cpu 
  147.69GB |  49.23GB | offload_optimizer=none
Estimated memory needed for params, optim states and gradients for a:
HW: Setup with 1 node, 8 GPUs per node.
SW: Model with 6607M total params.
  per CPU  |  per GPU |   Options
  2

## Deepspeed Stage3, 7B

In [9]:
from transformers import AutoModel
from deepspeed.runtime.zero.stage3 import estimate_zero3_model_states_mem_needs_all_live
model = AutoModel.from_pretrained("/work/u00cjz00/slurm_jobs/github/models/Llama-2-7b-chat-hf")
estimate_zero3_model_states_mem_needs_all_live(model, num_gpus_per_node=1, num_nodes=1)
estimate_zero3_model_states_mem_needs_all_live(model, num_gpus_per_node=1, num_nodes=2)
#estimate_zero3_model_states_mem_needs_all_live(model, num_gpus_per_node=1, num_nodes=4)
#estimate_zero3_model_states_mem_needs_all_live(model, num_gpus_per_node=1, num_nodes=8)
#estimate_zero3_model_states_mem_needs_all_live(model, num_gpus_per_node=4, num_nodes=2)
#estimate_zero3_model_states_mem_needs_all_live(model, num_gpus_per_node=8, num_nodes=2)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Estimated memory needed for params, optim states and gradients for a:
HW: Setup with 1 node, 1 GPU per node.
SW: Model with 6607M total params, 131M largest layer params.
  per CPU  |  per GPU |   Options
  166.15GB |   0.49GB | offload_param=cpu , offload_optimizer=cpu , zero_init=1
  166.15GB |   0.49GB | offload_param=cpu , offload_optimizer=cpu , zero_init=0
  147.69GB |  12.80GB | offload_param=none, offload_optimizer=cpu , zero_init=1
  147.69GB |  12.80GB | offload_param=none, offload_optimizer=cpu , zero_init=0
    0.73GB | 111.25GB | offload_param=none, offload_optimizer=none, zero_init=1
   36.92GB | 111.25GB | offload_param=none, offload_optimizer=none, zero_init=0
Estimated memory needed for params, optim states and gradients for a:
HW: Setup with 2 nodes, 1 GPU per node.
SW: Model with 6607M total params, 131M largest layer params.
  per CPU  |  per GPU |   Options
   83.07GB |   0.49GB | offload_param=cpu , offload_optimizer=cpu , zero_init=1
   83.07GB |   0.49GB | offlo